In [9]:
print(torch.__version__)

1.13.1+cu117


In [4]:
# # Converting a Segmentation Model via CoreML -- https://developer.apple.com/videos/play/tech-talks/10154

# ### Imports

import urllib
import torch
import torch.nn as nn
import torchvision
import json

from torchvision import transforms
import coremltools as ct
from PIL import Image


# ### Load Sample Model and Image

# Load model
model = torch.hub.load('pytorch/vision:v0.6.0', 'deeplabv3_resnet101', pretrained=True).eval()
# Load sample image
input_image = Image.open("dog_and_cat.jpg")
input_image = input_image.resize((224, 224))

# ### Image Preprocessing

to_tensor = transforms.ToTensor()
input_tensor = to_tensor(input_image)
input_batch = input_tensor.unsqueeze(0)


# ### Trace the Model with PyTorch

# ### Wrap the Model to Allow Tracing

class WrappedDeeplabv3Resnet101(nn.Module):
    
    def __init__(self):
        super(WrappedDeeplabv3Resnet101, self).__init__()
        self.model = torch.hub.load('pytorch/vision:v0.6.0', 'deeplabv3_resnet101', pretrained=True).eval()
    
    def forward(self, x):
        res = self.model(x)
        x = res["out"]
        return x


# ### Trace the Wrapped Model

traceable_model = WrappedDeeplabv3Resnet101().eval()
trace = torch.jit.trace(traceable_model, input_batch)

Using cache found in /home/oem/.cache/torch/hub/pytorch_vision_v0.6.0
Using cache found in /home/oem/.cache/torch/hub/pytorch_vision_v0.6.0


## Core ML

In [5]:
# ### Convert to Core ML 

# Define input
_input = ct.ImageType(
    name="input_1", 
    shape=input_batch.shape, 
    bias=[-0.485/0.229,-0.456/0.224,-0.406/0.225], 
    scale= 1./(255*0.226)
)

# Convert model
mlmodel = ct.convert(
    trace,
    inputs=[_input],
)


# ### Set the Model Metadata

labels_json = {"labels": ["background", "aeroplane", "bicycle", "bird", "board", "bottle", "bus", "car", "cat", "chair", "cow", "diningTable", "dog", "horse", "motorbike", "person", "pottedPlant", "sheep", "sofa", "train", "tvOrMonitor"]}

mlmodel.type = 'imageSegmenter'
mlmodel.user_defined_metadata['com.apple.coreml.model.preview.params'] = json.dumps(labels_json)


# ### Save the Model for Visualization

mlmodel.save("SegmentationModel.mlmodel")

Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 944/944 [00:02<00:00, 341.83 ops/s] 


In [6]:
from torch.utils.mobile_optimizer import optimize_for_mobile

## PyTorch Mobile no metal support

In [8]:
optimized_model = optimize_for_mobile(trace, backend='Metal')
optimized_model._save_for_lite_interpreter("deeplabv3_resnet101_scripted.ptl")

RuntimeError: 0 INTERNAL ASSERT FAILED at "../torch/csrc/jit/ir/alias_analysis.cpp":608, please report a bug to PyTorch. We don't have an op for metal_prepack::conv2d_prepack but it isn't a special case.  Argument types: Tensor, NoneType, int[], int[], int[], int, NoneType, NoneType, 

Candidates:

## CPU only

In [10]:
optimized_model = optimize_for_mobile(trace)
optimized_model._save_for_lite_interpreter("deeplabv3_resnet101_scripted.ptl")